# Data wragling

Limpiar, transformar (unificar, consolidar, normalizar ...) y enriquecer el dataset para objetivos posteriores.

In [53]:
import pandas as pd

In [54]:
# df = pd.read_csv('../Extract/elpais_2022_01_25.csv')
df = pd.read_csv('../Extract/eluniversal_2022_01_25.csv')
df

,body,title,url
0,Luis Fernando Sánchez lleva 15 años liderando ...,"[Video] Luis Fernando, el hombre que sueña con...",https://www.eluniversal.com.co/multimedia/vide...
1,"Edwin Quintana, quien ya compró su boleta para...","Luis Díaz, la ficha clave de la selección Colo...",https://www.eluniversal.com.co/deportes/luis-d...
2,La agencia AFP consultó esa lista este martes ...,Rusia incluye al opositor encarcelado Navalni ...,https://www.eluniversal.com.co/mundo/rusia-inc...
3,Cartagena fue noticia nacional la semana pasad...,Doble golpe a la institucionalidad,https://www.eluniversal.com.co/opinion/columna...
4,Al transitar por las vías interdepartamentales...,Bolívar: zona de riesgo electoral según la Moe...,https://www.eluniversal.com.co/politica/boliva...
...,...,...,...
91,El artista multiplatino Harry Styles anuncia l...,Harry Styles reprograma su gira y llega a Colo...,https://www.eluniversal.com.co/viernes/harry-s...
92,Los casos de COVID-19 en el departamento de Su...,Sucre registra más de 1.200 casos activos de C...,https://www.eluniversal.com.co/regional/sucre-...
93,"El tiempo no regresa, eso está claro, sin emba...","Jerónimo Cantillo, un rebelde con causa",https://www.eluniversal.com.co/viernes/jeronim...
94,Hace unos días noté consternado a un amigo y c...,Lo que dicen los demás,https://www.eluniversal.com.co/opinion/columna...


## Añadir el nombre del periodico al DataFrame

In [55]:
df['newspaper_uid'] = 'eluniversal'
df.head(3)

,body,title,url,newspaper_uid
0,Luis Fernando Sánchez lleva 15 años liderando ...,"[Video] Luis Fernando, el hombre que sueña con...",https://www.eluniversal.com.co/multimedia/vide...,eluniversal
1,"Edwin Quintana, quien ya compró su boleta para...","Luis Díaz, la ficha clave de la selección Colo...",https://www.eluniversal.com.co/deportes/luis-d...,eluniversal
2,La agencia AFP consultó esa lista este martes ...,Rusia incluye al opositor encarcelado Navalni ...,https://www.eluniversal.com.co/mundo/rusia-inc...,eluniversal


## Obtener el Host

In [56]:
from urllib.parse import urlparse

df['host'] = df['url'].apply(lambda url: urlparse(url).netloc)
df.head(3)

,body,title,url,newspaper_uid,host
0,Luis Fernando Sánchez lleva 15 años liderando ...,"[Video] Luis Fernando, el hombre que sueña con...",https://www.eluniversal.com.co/multimedia/vide...,eluniversal,www.eluniversal.com.co
1,"Edwin Quintana, quien ya compró su boleta para...","Luis Díaz, la ficha clave de la selección Colo...",https://www.eluniversal.com.co/deportes/luis-d...,eluniversal,www.eluniversal.com.co
2,La agencia AFP consultó esa lista este martes ...,Rusia incluye al opositor encarcelado Navalni ...,https://www.eluniversal.com.co/mundo/rusia-inc...,eluniversal,www.eluniversal.com.co


In [57]:
df['host'].unique()

array(['www.eluniversal.com.co'], dtype=object)

In [58]:
df['host'].value_counts()

www.eluniversal.com.co    96
Name: host, dtype: int64

Mi consulta **select()** de El Universal estubo tan buena que solo me devolvió los enlaces de El Universal XD. Pero El Pais sí devuelve enlaces de otros tipos.

## Datos faltantes

In [59]:
df.notna().sum()

body             96
title            96
url              96
newspaper_uid    96
host             96
dtype: int64

No tenía ningún faltante, entonces para las pruebas voy a generarlos

In [60]:
df.loc[:5,['title','url']]

,title,url
0,"[Video] Luis Fernando, el hombre que sueña con...",https://www.eluniversal.com.co/multimedia/vide...
1,"Luis Díaz, la ficha clave de la selección Colo...",https://www.eluniversal.com.co/deportes/luis-d...
2,Rusia incluye al opositor encarcelado Navalni ...,https://www.eluniversal.com.co/mundo/rusia-inc...
3,Doble golpe a la institucionalidad,https://www.eluniversal.com.co/opinion/columna...
4,Bolívar: zona de riesgo electoral según la Moe...,https://www.eluniversal.com.co/politica/boliva...
5,"Con operativos de control, autoridades se toma...",https://www.eluniversal.com.co/sucesos/con-ope...


In [61]:
df.loc[0,'title'] = pd.NA
df.loc[2,'title'] = pd.NA
print(df.loc[:5,'title'])

0                                                 <NA>
1    Luis Díaz, la ficha clave de la selección Colo...
2                                                 <NA>
3                   Doble golpe a la institucionalidad
4    Bolívar: zona de riesgo electoral según la Moe...
5    Con operativos de control, autoridades se toma...
Name: title, dtype: object


In [62]:
missing_titles_mask = df['title'].isna()

missing_titles = (df[missing_titles_mask]['url']
                     .str.extract(r'(?P<missing_titles>[^/]+)$')['missing_titles']
                     .str.replace('-',' ').str.capitalize()
                )
print(missing_titles.shape)
df.loc[missing_titles_mask, 'title'] = missing_titles
df.loc[:5,'title']

(2,)


0    Video luis fernando el hombre que suena con me...
1    Luis Díaz, la ficha clave de la selección Colo...
2    Rusia incluye al opositor encarcelado navalni ...
3                   Doble golpe a la institucionalidad
4    Bolívar: zona de riesgo electoral según la Moe...
5    Con operativos de control, autoridades se toma...
Name: title, dtype: object

## ID único a cada fila

In [63]:
import hashlib

In [64]:
df['url'][0].encode()

b'https://www.eluniversal.com.co/multimedia/video-luis-fernando-el-hombre-que-suena-con-mejorar-el-cano-juan-angola-BX5819470'

In [65]:
bytes(df['url'][0].encode())

b'https://www.eluniversal.com.co/multimedia/video-luis-fernando-el-hombre-que-suena-con-mejorar-el-cano-juan-angola-BX5819470'

In [66]:
hashlib.md5(bytes(df['url'][0].encode()))

<md5 _hashlib.HASH object @ 0x00000191F7058530>

In [67]:
hashlib.md5(bytes(df['url'][0].encode())).hexdigest()

'5ef9647a5e0c42cb0650128d16077edf'

In [68]:
uids = (
    df.apply(lambda row: hashlib.md5(bytes(row['url'].encode())).hexdigest(), axis='columns')
)
print(uids)
df['uid'] = uids
df.set_index('uid', inplace=True)
df.head(2)

0     5ef9647a5e0c42cb0650128d16077edf
1     72709636297ae012421efac327240580
2     ee078cbf9ad0f942a9ef35e1d8b251fd
3     0a47a3de2199cd935955a06d89b147af
4     8e58a14bd82417ee89673256e8700708
                    ...               
91    6c1bbdec7e142d75357dcaa7b4884d56
92    ba3651ca069af97d63fa00095653f7a8
93    5579cd5943d55e89dd519fc851f7d700
94    a8f6125e9b2f4c7831cbd2d567828ac8
95    3a81e8453470c7469a75d8be4b350d11
Length: 96, dtype: object


,body,title,url,newspaper_uid,host
uid,,,,,
5ef9647a5e0c42cb0650128d16077edf,Luis Fernando Sánchez lleva 15 años liderando ...,Video luis fernando el hombre que suena con me...,https://www.eluniversal.com.co/multimedia/vide...,eluniversal,www.eluniversal.com.co
72709636297ae012421efac327240580,"Edwin Quintana, quien ya compró su boleta para...","Luis Díaz, la ficha clave de la selección Colo...",https://www.eluniversal.com.co/deportes/luis-d...,eluniversal,www.eluniversal.com.co


## Caracteres basura

In [69]:
df['body'][0] = '\n' + df['body'][0]
df['body'][0]

'\nLuis Fernando Sánchez lleva 15 años liderando la Fundación Planeta Azul Caribe, que busca crear conciencia sobre el valor del caño Juan Angola y la necesidad de recuperarlo entre todos. (Lea aquí: El caño Juan Angola sí tiene dolientes)'

In [70]:
df['body'] = df['body'].str.replace('\n','')
df['body'][0]


'Luis Fernando Sánchez lleva 15 años liderando la Fundación Planeta Azul Caribe, que busca crear conciencia sobre el valor del caño Juan Angola y la necesidad de recuperarlo entre todos. (Lea aquí: El caño Juan Angola sí tiene dolientes)'

## Enriquecimiento de los datos

Tokenizar el título y el body

In [71]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [72]:
stop_words = stopwords.words('Spanish')
len(stop_words)

313

In [73]:
tokens = nltk.word_tokenize(df['title'][0])
print(tokens)

['Video', 'luis', 'fernando', 'el', 'hombre', 'que', 'suena', 'con', 'mejorar', 'el', 'cano', 'juan', 'angola', 'bx5819470']


In [74]:
tokens = list(filter(lambda token: token.isalpha(), tokens))
print(tokens)

['Video', 'luis', 'fernando', 'el', 'hombre', 'que', 'suena', 'con', 'mejorar', 'el', 'cano', 'juan', 'angola']


In [75]:
tokens = list(map(lambda token: token.lower(), tokens))
print(tokens)

['video', 'luis', 'fernando', 'el', 'hombre', 'que', 'suena', 'con', 'mejorar', 'el', 'cano', 'juan', 'angola']


In [76]:
tokens = list(filter(lambda token: token not in stop_words, tokens))
print(tokens)

['video', 'luis', 'fernando', 'hombre', 'suena', 'mejorar', 'cano', 'juan', 'angola']


In [77]:
def tokenize_column(df, column):
    return (
        df[column].dropna()
        .apply(lambda row: nltk.word_tokenize(row))
        .apply(lambda words: list(filter(lambda word: word.isalpha(), words)))
        .apply(lambda words: list(map(lambda word: word.lower(), words)))
        .apply(lambda words: list(filter(lambda word: word not in stop_words, words)))
        .apply(lambda words: len(words))
    )
column = 'title'
df['n_tokens_'+column] = tokenize_column(df, 'title')
df

,body,title,url,newspaper_uid,host,n_tokens_title
uid,,,,,,
5ef9647a5e0c42cb0650128d16077edf,Luis Fernando Sánchez lleva 15 años liderando ...,Video luis fernando el hombre que suena con me...,https://www.eluniversal.com.co/multimedia/vide...,eluniversal,www.eluniversal.com.co,9
72709636297ae012421efac327240580,"Edwin Quintana, quien ya compró su boleta para...","Luis Díaz, la ficha clave de la selección Colo...",https://www.eluniversal.com.co/deportes/luis-d...,eluniversal,www.eluniversal.com.co,7
ee078cbf9ad0f942a9ef35e1d8b251fd,La agencia AFP consultó esa lista este martes ...,Rusia incluye al opositor encarcelado navalni ...,https://www.eluniversal.com.co/mundo/rusia-inc...,eluniversal,www.eluniversal.com.co,8
0a47a3de2199cd935955a06d89b147af,Cartagena fue noticia nacional la semana pasad...,Doble golpe a la institucionalidad,https://www.eluniversal.com.co/opinion/columna...,eluniversal,www.eluniversal.com.co,3
8e58a14bd82417ee89673256e8700708,Al transitar por las vías interdepartamentales...,Bolívar: zona de riesgo electoral según la Moe...,https://www.eluniversal.com.co/politica/boliva...,eluniversal,www.eluniversal.com.co,8
...,...,...,...,...,...,...
6c1bbdec7e142d75357dcaa7b4884d56,El artista multiplatino Harry Styles anuncia l...,Harry Styles reprograma su gira y llega a Colo...,https://www.eluniversal.com.co/viernes/harry-s...,eluniversal,www.eluniversal.com.co,6
ba3651ca069af97d63fa00095653f7a8,Los casos de COVID-19 en el departamento de Su...,Sucre registra más de 1.200 casos activos de C...,https://www.eluniversal.com.co/regional/sucre-...,eluniversal,www.eluniversal.com.co,4
5579cd5943d55e89dd519fc851f7d700,"El tiempo no regresa, eso está claro, sin emba...","Jerónimo Cantillo, un rebelde con causa",https://www.eluniversal.com.co/viernes/jeronim...,eluniversal,www.eluniversal.com.co,4


## Datos duplicados

In [78]:
df.nunique()

body              93
title             93
url               96
newspaper_uid      1
host               1
n_tokens_title     8
dtype: int64

In [79]:
df['title'].value_counts()

El Curita, mantero cartagenero, fuera de peligro tras ser corneado                   2
Con operativos de control, autoridades se tomaron zona sur de Sincelejo              2
A Egan Bernal le tratan una infección y contusión pulmonar                           2
Video luis fernando el hombre que suena con mejorar el cano juan angola bx5819470    1
El Whatsapp destronó a las llamadas telefónicas                                      1
                                                                                    ..
OMS recomienda priorizar investigación de la versión BA.2 de ómicron                 1
Las formas y el fondo                                                                1
¿Qué es el síndrome del burnout?                                                     1
Nairo Quintana: “Siempre es posible levantarse, abrazo y fortaleza Egan”             1
Gobernador decreta nuevamente calamidad pública para la Mojana y el San Jorge        1
Name: title, Length: 93, dtype: int64

In [80]:
df[df['title'].duplicated()]

,body,title,url,newspaper_uid,host,n_tokens_title
uid,,,,,,
1e76f1cced96db7d9963e6603d7bede9,"El Curita, reconocido mantero oriundo de la ci...","El Curita, mantero cartagenero, fuera de pelig...",https://www.eluniversal.com.co/regional/el-cur...,eluniversal,www.eluniversal.com.co,7
d312c1d4d067fee335b44b62d352f22a,La Policía Nacional continúa realizado operati...,"Con operativos de control, autoridades se toma...",https://www.eluniversal.com.co/regional/con-op...,eluniversal,www.eluniversal.com.co,7
1b7a78c40b80f05a6ba485fdbab243b2,Tras la tercera cirugía a la que fue sometido ...,A Egan Bernal le tratan una infección y contus...,https://www.eluniversal.com.co/deportes/a-egan...,eluniversal,www.eluniversal.com.co,6


In [81]:
df[df['title'].str.contains('Curita')]['url']

uid
270f8c052c32a47a7b4b0aeb750bbe50    https://www.eluniversal.com.co/sucesos/el-curi...
1e76f1cced96db7d9963e6603d7bede9    https://www.eluniversal.com.co/regional/el-cur...
Name: url, dtype: object

In [82]:
df.drop_duplicates(subset=['title'], inplace=True)

In [83]:
df['title'].value_counts()

Video luis fernando el hombre que suena con mejorar el cano juan angola bx5819470    1
Zonas Francas, un instrumento para la competitividad                                 1
[Video] Entre la pobreza y la soledad, así es la vida de María Encarnación           1
EE. UU.: Examen de ingreso universitario ahora será digital                          1
Juicios por protestas: “En Cuba hay niños presos políticos”                          1
                                                                                    ..
El Curita, mantero cartagenero, fuera de peligro tras ser corneado                   1
¿Qué es el síndrome del burnout?                                                     1
Nairo Quintana: “Siempre es posible levantarse, abrazo y fortaleza Egan”             1
Carlos Mattos se encuentra en la Clínica Santa Fe por problemas de salud             1
Gobernador decreta nuevamente calamidad pública para la Mojana y el San Jorge        1
Name: title, Length: 93, dtype: int64

In [84]:
df.nunique()

body              93
title             93
url               93
newspaper_uid      1
host               1
n_tokens_title     8
dtype: int64